In [98]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import cv2
import PIL
import os
import sys
import glob
import random

from pprint import pprint
from ipywidgets import Video

from PIL import Image
from PIL.ExifTags import TAGS

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [99]:
def imshow(a):
  a = a.clip(0, 255).astype('uint8')
  if a.ndim == 3:
    if a.shape[2] == 4:
      a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
    else:
      a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)
  display(PIL.Image.fromarray(a))

In [100]:
!mkdir data

from pydrive.auth import GoogleAuth
from google.colab import drive
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
drive.CreateFile({"id": "1APJsddWLOTyYxlgK-nHNK9o6AU_B-NgY"}).GetContentFile('./data/test.mp4')

mkdir: cannot create directory ‘data’: File exists


In [ ]:
# 1 - poszukajmy wgl stołu XD
video = cv2.VideoCapture("./data/test.mp4")

video_width = int(video.get(3))
video_height = int(video.get(4))
video_fps = video.get(cv2.CAP_PROP_FPS)

if not video.isOpened():
    print("Error while opened the film.")

video_track = cv2.VideoWriter("./data/result.avi",  cv2.VideoWriter_fourcc(*'DIVX'), video_fps, (video_width, video_height))
for _ in range(200):
    ret, frame = video.read()

    if ret:
        dst = cv2.Canny(frame, 50, 200, None, 3)
        table_borders = cv2.HoughLinesP(dst, 1, np.pi / 180, 150, None, 400, 50)

        for x_begin, y_begin, x_end, y_end in table_borders[:, 0, :]:
            cv2.line(frame, (x_begin, y_begin), (x_end, y_end), (0, 0, 255), 3)

        video_track.write(frame)
    else:
        break

video_track.release()
!ffmpeg -hide_banner -loglevel error -i ./data/result.avi -y ./data/result.mp4
Video.from_file("./data/result.mp4")

In [ ]:
# 2 - wyciecie stołu na sztywno bo czemu nie
video = cv2.VideoCapture("./data/test.mp4")

video_width = int(video.get(3))
video_height = int(video.get(4))
video_fps = video.get(cv2.CAP_PROP_FPS)

if not video.isOpened():
    print("Error while opened the film.")

video_track = cv2.VideoWriter("./data/result-balls.avi",  cv2.VideoWriter_fourcc(*'DIVX'), video_fps, (video_width, video_height))
for _ in range(200):
    ret, frame = video.read()

    HA, HB = (320, 195), (970, 195)
    HC, HD = (200, 630), (1100, 630)

    if ret:
        H, W = frame.shape[:2]
        mask = np.zeros((H, W), dtype=np.uint8)
        
        cv2.fillConvexPoly(mask, np.array([HA, HC, HD, HB]), 1)

        video_track.write(frame * np.dstack([mask]*3))
    else:
        break

video_track.release()
!ffmpeg -hide_banner -loglevel error -i ./data/result-balls.avi -y ./data/result-balls.mp4
Video.from_file("./data/result-balls.mp4")

In [ ]:
# 3 - detekcja piły ze stołem ustawionym na razie na sztywno
video = cv2.VideoCapture("./data/result-balls.mp4")

video_width = int(video.get(3))
video_height = int(video.get(4))
video_fps = video.get(cv2.CAP_PROP_FPS)

if not video.isOpened():
    print("Error while opened the film.")

video_track = cv2.VideoWriter("./data/result-balls-detected.avi",  cv2.VideoWriter_fourcc(*'DIVX'), video_fps, (video_width, video_height))
for _ in range(200):
    ret, frame = video.read()

    if ret:
      frame = cv2.GaussianBlur(frame, (3, 3), 0)
      frame_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

      circles = cv2.HoughCircles(frame_gray, cv2.HOUGH_GRADIENT, 1, 5, param1=60, param2=15, minRadius=0, maxRadius=20)
      circles = np.uint16(np.around(circles))
      for i in circles[0, :]:
          cv2.circle(frame,(i[0],i[1]),i[2],(0,255,0),2)
          cv2.circle(frame,(i[0],i[1]),2,(0,0,255),3)
      video_track.write(frame)
    else:
        break

video_track.release()
!ffmpeg -hide_banner -loglevel error -i ./data/result-balls-detected.avi -y ./data/result-balls-detected.mp4
Video.from_file("./data/result-balls-detected.mp4")